In [1]:
import pandas as pd
import numpy as np
import random
import string
from datetime import datetime, timedelta

from pyspark.sql.functions import coalesce, lit, rand
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, MinMaxScaler
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier, NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegressionModel, RandomForestClassificationModel
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

In [2]:
packages = [
    'org.apache.hadoop:hadoop-aws:3.3.4',
    'org.apache.hadoop:hadoop-client-api:3.3.4',
    'org.apache.hadoop:hadoop-client-runtime:3.3.4',
]

conf = SparkConf() \
    .setAppName("MyApp") \
    .set("spark.driver.memory", "8g") \
    .set("spark.executor.memory", "8g") \
    .set('spark.jars.packages', ','.join(packages))

sc = SparkContext(conf=conf)

hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
hadoop_config.set('com.amazonaws.services.s3.enableV4', 'true')

spark = SparkSession(sc)

24/07/20 02:26:29 WARN Utils: Your hostname, thamaMBP.local resolves to a loopback address: 127.0.0.1; using 172.20.10.5 instead (on interface en0)
24/07/20 02:26:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/thama/ghq/github.com/mathmathpp2/synapselightgbm_trial/synapse-test/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/thama/.ivy2/cache
The jars for the packages stored in: /Users/thama/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client-api added as a dependency
org.apache.hadoop#hadoop-client-runtime added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fcdcdc27-d263-49fa-a193-e03caf24d779;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.slf4j#slf4j-api;1.7.36 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.2 in local-m2-cache
:: resolution report :: resolve 496ms :: artifacts dl 24ms
	:: modul

In [3]:
spark

In [4]:
def random_date(start, end):
    return start + timedelta(days=random.randint(0, int((end - start).days)))

def random_geohash():
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=11))

data = []
start_date = datetime(2024, 7, 1)
end_date = datetime(2024, 7, 4)
colors = ["red", "blue", "green"]
types = ["SUV", "sedan", "truck"]

for _ in range(1000):
    id = _
    color = random.choice(colors)
    type = random.choice(types)
    hour = random.randint(0, 4)
    milesperhour = random.uniform(19.0, 35.0)
    age = random.randint(40, 85)

    if random.randint(1, 10) < 7:
        if color == "red":
            label = 1
        elif color == "blue":
            label = 2
        else:
            label = 3
    else:
        label = random.randint(1, 3)
    
    date = random_date(start_date, end_date).strftime("%Y-%m-%d")
    geohash = random_geohash()
    data.append((id, color, type, hour, milesperhour, age, label, date, geohash))

schema = ["id", "color", "type", "hour", "milesperhour", "age", "label", "date", "geohash"]
data = spark.createDataFrame(data, schema)

data.show()

+---+-----+-----+----+------------------+---+-----+----------+-----------+
| id|color| type|hour|      milesperhour|age|label|      date|    geohash|
+---+-----+-----+----+------------------+---+-----+----------+-----------+
|  0| blue|sedan|   1|34.082181961217046| 48|    3|2024-07-01|24i48e9i4j1|
|  1|  red|sedan|   0|23.838102366558587| 83|    1|2024-07-02|7tkx93qn8z9|
|  2| blue|truck|   2|32.368643573261394| 85|    1|2024-07-03|97sgui0463f|
|  3| blue|truck|   4|23.554232985756123| 65|    3|2024-07-02|yb3w6c9km7w|
|  4|green|  SUV|   3|27.497758222648912| 50|    3|2024-07-04|kcwv1lmmcrc|
|  5| blue|  SUV|   4|28.390148083816293| 72|    2|2024-07-02|t758bkka16i|
|  6|green|  SUV|   2| 30.62108528150685| 42|    3|2024-07-02|2h79c7blsjn|
|  7|green|  SUV|   1|33.295822665972786| 81|    3|2024-07-02|euwjxkr18v6|
|  8|  red|  SUV|   1|20.431989967799687| 63|    1|2024-07-04|kcig7tidkwo|
|  9|  red|  SUV|   1| 19.58002103070639| 55|    1|2024-07-04|dblwpbpfrio|
| 10|  red|truck|   4| 25

In [8]:
# string label
# data = spark.createDataFrame([
#     (0, "red",   "SUV",   12, 20.0, 60, "class1", "2024-07-01", "u4pruydqqvj"),
#     (1, "red",   "sedan",  9, 30.0, 70, "class2", "2024-07-02", "u4pruydqqvk"),
#     (2, "red",   "truck", 15, 25.0, 80, "class3", "2024-07-01", "u4pruydqqvj"),
#     (3, "blue",  "SUV",   20, 22.0, 65, "class1", "2024-07-02", "u4pruydqqvk"),
#     (4, "blue",  "sedan",  5, 35.0, 75, "class1", "2024-07-01", "u4pruydqqvj"),
#     (5, "blue",  "truck", 12, 28.0, 85, "class3", "2024-07-02", "u4pruydqqvk"),
#     (6, "green", "SUV",    9, 19.0, 50, "class2", "2024-07-03", "u4pruydqqvl"),
#     (7, "green", "sedan", 15, 32.0, 60, "class3", "2024-07-03", "u4pruydqqvm"),
#     (8, "green", "truck", 20, 27.0, 40, "class1", "2024-07-04", "u4pruydqqvn"),
#     (9, "green", "SUV",    5, 21.0, 55, "class2", "2024-07-04", "u4pruydqqvo")
# ], ["id", "color", "type", "hour", "milesperhour", "age", "label", "date", "geohash"])

# Split the data into training and test sets (80% training, 20% test)
train_data, test_data = data.randomSplit([0.8, 0.2], seed=1234)

# String Indexing for features
indexers = [
    StringIndexer(inputCol="label", outputCol="label_index"), # !!!
    StringIndexer(inputCol="color", outputCol="color_index"),
    StringIndexer(inputCol="type", outputCol="type_index"),
    StringIndexer(inputCol="hour", outputCol="hour_index")
]

# One-Hot Encoding for Logistic Regression (not needed for tree-based models)
encoder = OneHotEncoder(
    inputCols=["color_index", "type_index", "hour_index"],
    outputCols=["color_vec", "type_vec", "hour_vec"]
)

# Assembling Features for Logistic Regression and Naive Bayes
assembler_lr_nb = VectorAssembler(
    inputCols=["color_vec", "type_vec", "hour_vec", "milesperhour", "age"],
    outputCol="assembled_features"
)

# Assembling Features for Tree-Based Models
assembler_tree = VectorAssembler(
    inputCols=["color_index", "type_index", "hour_index", "milesperhour", "age"],
    outputCol="features"
)

# MinMaxScaler for Logistic Regression and Naive Bayes
scaler_lr_nb = MinMaxScaler(inputCol="assembled_features", outputCol="features")

# Create and Fit the Pipeline for Logistic Regression and Naive Bayes
pipeline_lr_nb = Pipeline(stages=indexers + [encoder, assembler_lr_nb, scaler_lr_nb])
model_lr_nb = pipeline_lr_nb.fit(train_data)
transformed_train_data_lr_nb = model_lr_nb.transform(train_data)
transformed_test_data_lr_nb = model_lr_nb.transform(test_data)

# Create and Fit the Pipeline for Tree-Based Models
pipeline_tree = Pipeline(stages=indexers + [assembler_tree])
model_tree = pipeline_tree.fit(train_data)
transformed_train_data_tree = model_tree.transform(train_data)
transformed_test_data_tree = model_tree.transform(test_data)

In [9]:
model_lr_nb.stages[0].labels

['blue', 'green', 'red']

In [10]:
def get_features(df):
    if 'assembled_features' in df.schema.names:
        feature_attrs = df.schema['assembled_features'].metadata['ml_attr']['attrs']
    else:
        feature_attrs = df.schema['features'].metadata['ml_attr']['attrs']

    features = []
    for attr_type, attrs in feature_attrs.items():
        features += attrs

    # for each in sorted(features, key=lambda x: x['idx']):
    #     print(each['idx'], each['name'])
    
    feature_names = [each['name'] for each in sorted(features, key=lambda x: x['idx'])]

    return feature_names

In [11]:
get_features(transformed_train_data_lr_nb)

['color_vec_blue',
 'color_vec_green',
 'type_vec_sedan',
 'type_vec_truck',
 'hour_vec_1',
 'hour_vec_0',
 'hour_vec_4',
 'hour_vec_3',
 'milesperhour',
 'age']

In [12]:
get_features(transformed_train_data_tree)

['color_index', 'type_index', 'hour_index', 'milesperhour', 'age']

# Logistic regression

## save and load data

In [13]:
# save to local
data_path = "data/transformed_train_data_lr_nb/"

# save to s3
# data_path = "s3a://test-thama-misc-20210612/20240717-sparkml/data/transformed_train_data_lr_nb/"

transformed_train_data_lr_nb.write.partitionBy("date", "geohash").mode('overwrite').save(data_path)

In [14]:
# load data
transformed_train_data_lr_nb_loaded = spark.read.load(data_path)

# print
transformed_train_data_lr_nb_loaded.toPandas().head(1)

,id,color,type,hour,milesperhour,age,label,color_index,type_index,hour_index,color_vec,type_vec,hour_vec,assembled_features,features,date,geohash
0,786,green,sedan,1,23.314242,83,3,1.0,0.0,0.0,"(0.0, 1.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 23.31...","(0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.269...",2024-07-02,j2jpaekvi54


## save and load pipeline

In [15]:
# save to local
pipeline_model_path = "pipelines/model_lr_nb"

# save to s3
# pipeline_model_path = "s3a://test-thama-misc-20210612/20240717-sparkml/pipelines/model_lr_nb"

model_lr_nb.write().overwrite().save(pipeline_model_path)

In [16]:
# load pipeline
loaded_model = PipelineModel.load(pipeline_model_path)

## create model

In [17]:
# Training the Logistic Regression Model
lr = LogisticRegression(featuresCol="features", labelCol="label_index") # !!!
lr_model = lr.fit(transformed_train_data_lr_nb)

24/07/20 02:27:41 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


## save and load model

In [18]:
# save to local
model_path = "models/lr_model/"

# save to s3
# model_path = "s3a://test-thama-misc-20210612/20240717-sparkml/models/lr_model/"

lr_model.write().overwrite().save(model_path)

In [19]:
# load model
lr_model = LogisticRegressionModel.load(model_path)

## interpret model

In [20]:
# Get model coefficients and intercept for Logistic Regression
coefficients = lr_model.coefficientMatrix
intercept = lr_model.interceptVector
print(f"Coefficients: {coefficients}")
print(f"Intercept: {intercept}")

Coefficients: DenseMatrix([[ 0.09863039,  0.6373956 ,  0.04769662, -0.13441972, -0.1233593 ,
               0.05768763, -0.16004556,  0.16768973,  0.03963142,  0.08391418],
             [-1.54615485, -1.67036284, -0.05615882, -0.07673807,  0.06647786,
               0.1301485 ,  0.22368808, -0.1703098 , -0.15508396,  0.13825928],
             [ 1.64706143, -0.5214493 ,  0.03806322,  0.05313225, -0.18434929,
              -0.20192988, -0.24674252, -0.31196386,  0.17187505,  0.07875167],
             [-0.19953696,  1.55441653, -0.02960102,  0.15802554,  0.24123073,
               0.01409376,  0.18309999,  0.31458394, -0.05642252, -0.30092513]])
Intercept: [-16.472841777995686,6.569559211345696,5.002800057733383,4.900482508916609]


In [21]:
# shape = num_classes x num_features
np.array(coefficients.toArray().tolist()).shape

(4, 10)

In [23]:
coef_df = pd.DataFrame(
    np.array(coefficients.toArray().tolist()),
    columns=get_features(transformed_train_data_lr_nb)
)

# map label_index to label
coef_df['label'] = loaded_model.stages[0].labels
coef_df[['label'] + list(coef_df.columns)[:-1]]

ValueError: Length of values (3) does not match length of index (4)

In [ ]:
coef_df2 = coef_df.reset_index().melt(id_vars=['label'], var_name='feature', value_name='coefficient')
coef_df2[['label', 'feature', 'coefficient']].sort_values(by=['label', 'feature'])

## evaluate

In [25]:
# # Initialize evaluators for all models
# evaluator_accuracy = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label_index", metricName="accuracy")
# evaluator_precision = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label_index", metricName="weightedPrecision")
# evaluator_recall = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label_index", metricName="weightedRecall")
# evaluator_f1 = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label_index", metricName="f1")

# Initialize evaluators for all models
evaluator_accuracy = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label", metricName="accuracy")
evaluator_precision = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label", metricName="weightedPrecision")
evaluator_recall = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label", metricName="weightedRecall")
evaluator_f1 = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label", metricName="f1")

In [26]:
# Evaluating the Logistic Regression Model
lr_predictions = lr_model.transform(transformed_test_data_lr_nb)
lr_accuracy = evaluator_accuracy.evaluate(lr_predictions)
lr_precision = evaluator_precision.evaluate(lr_predictions)
lr_recall = evaluator_recall.evaluate(lr_predictions)
lr_f1 = evaluator_f1.evaluate(lr_predictions)
print(f"Logistic Regression Accuracy: {lr_accuracy}")
print(f"Logistic Regression Precision: {lr_precision}")
print(f"Logistic Regression Recall: {lr_recall}")
print(f"Logistic Regression F1 Score: {lr_f1}")

Logistic Regression Accuracy: 0.7014218009478673
Logistic Regression Precision: 0.7123299902820629
Logistic Regression Recall: 0.7014218009478672
Logistic Regression F1 Score: 0.702552651112263


# Random forest

## save and load data

In [75]:
# save to local
data_path = "data/transformed_train_data_tree/"

# save to s3
# data_path = "s3a://test-thama-misc-20210612/20240717-sparkml/data/transformed_train_data_tree/"

transformed_train_data_tree.write.partitionBy("date", "geohash").mode('overwrite').save(data_path)

In [76]:
# load
transformed_train_data_tree_loaded = spark.read.load(data_path)

## save and load pipeline

In [77]:
# save to local
pipeline_model_path = "pipelines/model_tree"

# save to s3
# pipeline_model_path = "s3a://test-thama-misc-20210612/20240717-sparkml/pipelines/model_tree"

model_tree.write().overwrite().save(pipeline_model_path)

In [78]:
# load pipeline
loaded_model = PipelineModel.load(pipeline_model_path)

## create model

In [79]:
# Training the Random Forest Classifier
rf = RandomForestClassifier(featuresCol="features", labelCol="label_index")
rf_model = rf.fit(transformed_train_data_tree)

24/07/19 10:07:21 WARN DecisionTreeMetadata: DecisionTree reducing maxBins from 32 to 9 (= number of training instances)


## save and load model

In [80]:
# save to local
model_path = "models/rf_model/"

# save to s3
# model_path = "s3a://test-thama-misc-20210612/20240717-sparkml/models/rf_model/"

rf_model.write().overwrite().save(model_path)

In [81]:
# load model
rf_model = RandomForestClassificationModel.load(model_path)

## interpret model

In [82]:
# Get feature importances for Random Forest
rf_feature_importances = rf_model.featureImportances.toArray()
features_importances_rf = [(assembler_tree.getInputCols()[i], float(rf_feature_importances[i])) for i in range(len(rf_feature_importances))]
importances_df_rf = pd.DataFrame(features_importances_rf, columns=["Feature", "Importance"]).sort_values(by='Importance', ascending=False)

In [83]:
importances_df_rf

,Feature,Importance
0,color_index,0.301543
3,milesperhour,0.298617
2,hour_index,0.258801
1,type_index,0.075918
4,age,0.065121
